In [41]:
import pandas as pd
import glob
import warnings
import os
import requests
import gspread
from dotenv import load_dotenv

load_dotenv()
base_id = "appEX0yBBP1xhAkeL"
api_key = "keyedN52o4Usdo8DS"
table_name = 'USERS'
AIRTABLE_BASE_ID=os.environ.get(base_id)
AIRTABLE_API_KEY=os.environ.get(api_key)
AIRTABLE_TABLE_NAME=os.environ.get(table_name)
AIRTABLE_URL = f"https://api.airtable.com/v0{AIRTABLE_BASE_ID}"
warnings.filterwarnings('ignore')

# Archivo Google Sheets
sa = gspread.service_account(filename = './gspread/service_account.json')
sh = sa.open('Users TUDUU')
wks = sh.worksheet('automatizacion')

json = wks.get_all_records()
df = pd.DataFrame(json)

load_dotenv()

path = r'cluster_2_1/' # use your path
#save into df all files
all_files = glob.glob(path + "/*.xlsx")

In [43]:
# Para conectarnos a AirTable

#!pip install pyairtable
#!pip airtable install airtable-python-wrapper
from pyairtable import Table
table = Table(api_key, base_id, table_name)

**Clusters**:
* Como muy sano y hago deporte -> 1
* Dieta equilibrada, sin excesos -> 2
* Como sano pero me gusta caer en la tentación… -> 3
* No me quito nada, como de todo -> 4


In [110]:
tipos_consumidores = pd.DataFrame(pd.DataFrame(table.all(fields = [' Email', 'Tipo de consumidor']))['fields'].tolist())

In [112]:
# los clusters los deciden los propios clientes a la hora de hacer el test
import numpy as np
tipos_consumidores['cluster'] = ''
tipos_consumidores['cluster']  = np.where(tipos_consumidores['Tipo de consumidor']=='Como muy sano y hago deporte', str(1), tipos_consumidores['cluster'])
tipos_consumidores['cluster']  = np.where(tipos_consumidores['Tipo de consumidor']=='Dieta equilibrada, sin excesos', str(2), tipos_consumidores['cluster'])
tipos_consumidores['cluster']  = np.where(tipos_consumidores['Tipo de consumidor']=='Como sano pero me gusta caer en la tentación', str(3), tipos_consumidores['cluster'])
tipos_consumidores['cluster']  = np.where(tipos_consumidores['Tipo de consumidor']=='No me quito nada, como de todo', str(4), tipos_consumidores['cluster'])
# Ahora hay muchos nan porque al registrarnos no hemos hecho el test, con los registros reales no 
# debería de haber ningún nan; todos tienen que hacer el test
# Los de prueba los metemos en el cluster 4
tipos_consumidores['cluster'].fillna(str(4))

0      
1      
2      
3      
4      
     ..
83     
84     
85     
86     
87     
Name: cluster, Length: 88, dtype: object

In [ ]:
# hasta aquí hemos llegado a conseguir la información del tipo de consumidor; ahora, tendríamos que conseguir comprobar qué usuario es el que ha entrado nuevo, es decir, en qué usuario detecta el "change" y coger el tipo de consumidor que le pertenece a ese usuario.

In [113]:
li = dict()
for filename in all_files:
    df = pd.read_excel(filename, index_col=None, header=0)
    filename = filename.split('/')[1].split('.')[0]
    df['prob'] = 1/len(df)
    li[filename] = df

In [114]:

def is_in_list(a, b):
    a = a.split(' ')
    a = [word.lower() for word in a]
    return any(i in a for i in b)


def set_probabilities(tipo_cluster):
    for i in range(len(li['Desayuno'] )):
        if is_in_list(li['Desayuno'][' Name'][i], tipo_cluster[0]):
            li['Desayuno']['prob'][i] *= 100
    for i in range(len(li['comida_cena'] )):
        if is_in_list(li['comida_cena'][' Name'][i], tipo_cluster[1]):
            li['comida_cena']['prob'][i] *= 100
    for i in range(len(li['entre_horas'] )):
        if is_in_list(li['entre_horas'][' Name'][i], tipo_cluster[2]):
            li['entre_horas']['prob'][i] *= 100
    Desayuno = li['Desayuno']
    comida_cena = li['comida_cena']
    entre_horas = li['entre_horas']
    Cuidado_personal = li['Cuidado_personal']
    Hogar = li['Hogar']
    return Desayuno, comida_cena, entre_horas, Cuidado_personal, Hogar



In [115]:
probx3_2_1_desayuno = ['cereales', 'pan', 'tostada', 'palitos', 'baguette', 'hogaza', 'baguettes', 'panadería', 'cereales','aceite',  'tostadas','galletas','tortitas', 'leche', 'cerezas', 'manzana', 'plátano', 'fresa', 'higos', 'paraguayo', 'nectarina','manzana', 'aguacates', 'mango', 'aguacate',  'limón', 'melón', 'sandía', 'kiwis', 'bífidus', 'queso', 'yogur', 'mantequilla']
probx3_2_1_comida_cena = ['huevo','huevos','tortiglioni','helice','tallarín','cannelloni','fusilli','pasta', 'macarrón', 'tallarines', 'garbanzos', 'lentejas', 'alubias','albóndigas','entrecot','burguer','solomillo','salchichas','bacalao','mejillones','almejas', 'bonito','langostino','atún', 'rodaballo', 'rape', 'merluza','gallo', 'dorada', 'lubina','lenguado', 'calamar','sepia','gambón','filete','cebolla', 'ajo','zanahoria', 'albahaca', 'calabaza','salmorejo','sopa', 'gazpacho', 'patatas', 'patata', 'arroz', 'tomate', 'pollo,','proteínas', 'pollo', 'salmón','arroz','tiramisú','mousse','flan', 'helado','ensalada','champiñones','pimientos', 'espinacas','cebolla','lechuga', 'iceberg','zanahoria', 'brócoli', 'berenjena','berenjenas', 'coliflor', 'pepino','alcachofa','tomate', 'puerro', 'calabaza', 'calabacín','pimiento','patata', 'remolacha', 'zanahoria', 'cebolla', 'ajo','ajos','puerro','espárrago','canónigos', 'coliflor','champiñón', 'seta','zanahorias', 'tomates']
probx3_2_1_entre_horas = [ 'chocolate', 'galletas','cheesecake', 'copa','brownie','cacao', 'choco', 'berlina', 'gofres', 'sobaos', 'croissant','snack','chocolate', 'pizza', 'tarta', 'hamburguesa','mayonesa', 'nata', 'paté','nachos','york','quesos','morcilla', 'ibéricos', 'ibérico','jamón', 'chorizo','paletilla', 'ibérica', 'salchichón', 'longaniza','paleta','lonchas', 'pavo', 'fiambre','lonchas']
cluster1 = [probx3_2_1_desayuno, probx3_2_1_comida_cena, probx3_2_1_entre_horas]

prob_3_2_2_desayuno = ['cereales','pan', 'tostada', 'palitos', 'baguette','hogaza','baguettes', 'panadería','cereales','aceite',  'tostadas','galletas','leche','cerezas','albaricoque', 'manzana', 'plátano', 'fresa','uvas','ciruela', 'higos', 'paraguayo','naranja', 'mandarina', 'uva', 'peras', 'pera', 'piña','manzanas', 'manzana', 'banana', 'ciruelas', 'paraguayos', 'frambuesas', 'aguacates', 'mango', 'aguacate','limón', 'melón', 'sandía', 'kiwis','bífidus','yogur','cereales', 'pan', 'tostada', 'palitos', 'baguette', 'hogaza', 'baguettes', 'panadería', 'cereales','aceite',  'tostadas','galletas','tortitas', 'leche', 'cerezas', 'manzana', 'plátano', 'fresa', 'higos', 'paraguayo', 'nectarina','manzana', 'aguacates', 'mango', 'aguacate',  'limón', 'melón', 'sandía', 'kiwis', 'bífidus', 'queso', 'yogur', 'mantequilla','zumo','leche']
prob_3_2_2_comida_cena = ['huevo','huevos','tortiglioni','helice','tallarín','cannelloni','fusilli','pasta', 'spaghetti','macarrón', 'tallarines','garbanzo','legumbre', 'garbanzos', 'lentejas', 'alubias', 'lenteja', 'alubia', 'lomo', 'cerdo', 'pollo', 'pechuga','solomillo', 'bonito', 'atún', 'rodaballo', 'rape', 'merluza','gallo', 'dorada', 'lubina','lenguado', 'calamar','filete','cebolla', 'ajo', 'espinaca', 'alcachofa', 'zanahoria', 'albahaca', 'calabaza', 'remolacha', 'salmorejo','sopa', 'gazpacho','patatas', 'patata', 'arroz', 'tomate','pollo,','proteínas', 'pollo', 'salmón','arroz','tiramisú','mousse', 'flan', 'helado','ensalada','alcachofas','champiñones','alcachofa','verduras','verdura', 'pimientos', 'espinacas', 'cebolla','haba', 'rollitos', 'rodajas', 'lechuga', 'iceberg', 'zanahoria', 'brócoli', 'berenjena','judía','berenjenas', 'coliflor', 'pepino', 'escarola', 'alcachofa', 'tomate', 'puerro', 'calabaza', 'calabacín', 'pimiento', 'patata', 'remolacha', 'zanahoria', 'cebolla', 'ajo','ajos','puerro', 'maíz','remolacha', 'espárrago', 'puerro', 'espinaca','rúcula', 'lechuga', 'canónigos', 'coliflor', 'brócoli', 'alcachofa','acelgas','champiñón', 'seta', 'zanahorias', 'tomates', 'cebollas', 'batata', 'lechugas']
prob_3_2_2_entre_horas =  ['chocolate', 'galletas','cheesecake', 'copa','brownie','cacao', 'choco', 'berlina', 'trenza', 'napolitana', 'palmeritas', 'mantecadas','gofres', 'sobaos','croissant','snack','chocolate', 'pizza', 'tarta', 'hamburguesa','mayonesa', 'nata','paté','nachos','york','quesos','morcilla', 'ibéricos', 'ibérico','jamón', 'chorizo','chistorra', 'paletilla', 'ibérica', 'salchichón', 'longaniza','paleta','lonchas', 'pavo', 'fiambre','lonchas']
cluster2 = [prob_3_2_2_desayuno, prob_3_2_2_comida_cena, prob_3_2_2_entre_horas]

prob_3_2_3_desayuno = ['cereales','pan', 'tostada', 'palitos','baguette','hogaza', 'baguettes', 'panadería','cereales','aceite',  'tostadas','leche','cerezas','albaricoque','manzana', 'plátano', 'fresa','uvas','ciruela', 'higos', 'paraguayo', 'arándanos', 'nectarina', 'melocotón', 'naranja', 'mandarina', 'uva', 'peras', 'pera', 'piña','manzanas', 'manzana', 'banana', 'ciruelas', 'paraguayos', 'frambuesas', 'aguacates', 'mango', 'aguacate', 'limón', 'melón', 'sandía', 'kiwis','bífidus','yogur','cereales', 'pan', 'tostada', 'palitos', 'baguette', 'hogaza', 'baguettes', 'panadería', 'cereales','aceite',  'tostadas','galletas','tortitas', 'leche', 'cerezas', 'manzana', 'plátano', 'fresa', 'higos', 'paraguayo', 'nectarina','manzana', 'aguacates', 'mango', 'aguacate',  'limón', 'melón', 'sandía', 'kiwis', 'bífidus', 'queso', 'yogur', 'zumo','leche']
prob_3_2_3_comida_cena = ['huevo','huevos','tortiglioni','helice','tallarín','cannelloni','fusilli','pasta', 'spaghetti', 'macarrón', 'tallarines', 'garbanzo','legumbre', 'garbanzos', 'lentejas', 'alubias', 'lenteja', 'alubia','pollo','pechuga','bonito', 'atún', 'rodaballo', 'rape', 'merluza','gallo', 'dorada', 'lubina', 'lenguado', 'calamar','filete','cebolla', 'ajo', 'espinaca', 'alcachofa', 'zanahoria', 'albahaca', 'calabaza', 'remolacha','salmorejo','sopa', 'gazpacho', 'patatas', 'patata', 'arroz', 'tomate','pollo,','proteínas', 'pollo', 'integrales','tiramisú','mousse', 'flan', 'helado','ensalada','alcachofas','champiñones','alcachofa','verduras','verdura', 'pimientos', 'espinacas', 'cebolla','haba', 'rollitos', 'rodajas', 'lechuga', 'iceberg','zanahoria', 'brócoli', 'berenjena','judía','berenjenas', 'coliflor', 'pepino', 'escarola', 'alcachofa', 'tomate', 'puerro', 'calabaza', 'calabacín','pimiento','patata', 'remolacha', 'zanahoria', 'cebolla', 'ajo','ajos','puerro','maíz','remolacha','espárrago', 'puerro','espinaca','rúcula', 'lechuga', 'canónigos', 'coliflor', 'brócoli', 'alcachofa','acelgas','champiñón', 'seta','zanahorias', 'tomates', 'cebollas', 'batata', 'lechugas']
prob_3_2_3_entre_horas = ['york','quesos','ibéricos', 'ibérico','jamón','paletilla', 'ibérica','paleta', 'lonchas', 'pavo','lonchas']
cluster3 = [prob_3_2_3_desayuno, prob_3_2_3_comida_cena, prob_3_2_3_entre_horas]

prob_3_2_4_desayuno = ['cereales','pan', 'tostada', 'palitos','baguette','hogaza','baguettes', 'panadería','cereales','aceite',  'tostadas', 'leche','muesli','cerezas','albaricoque','manzana', 'plátano', 'fresa','uvas','ciruela', 'higos', 'paraguayo', 'arándanos', 'nectarina', 'melocotón', 'naranja', 'mandarina', 'uva', 'peras', 'pera', 'piña', 'manzanas', 'manzana', 'banana', 'ciruelas', 'paraguayos', 'frambuesas','aguacates', 'mango', 'aguacate', 'limón', 'melón', 'sandía', 'kiwis','bífidus','yogur','kéfir','cereales', 'pan', 'tostada', 'palitos', 'baguette', 'hogaza', 'baguettes', 'panadería', 'cereales','aceite',  'tostadas','galletas','tortitas', 'leche', 'cerezas', 'manzana', 'plátano', 'fresa', 'higos', 'paraguayo', 'nectarina','manzana', 'aguacates', 'mango', 'aguacate',  'limón', 'melón', 'sandía', 'kiwis', 'bífidus', 'queso', 'yogur','zumo','leche']
prob_3_2_4_comida_cena = ['huevo','huevos','tortiglioni','helice','tallarín','cannelloni','fusilli','pasta', 'spaghetti', 'macarrón', 'tallarines','garbanzo','legumbre', 'garbanzos', 'lentejas', 'alubias', 'lenteja', 'alubia','pollo','pechuga','bonito', 'atún', 'rodaballo', 'rape', 'merluza','gallo', 'dorada', 'lubina', 'lenguado', 'calamar','filete','cebolla', 'ajo', 'espinaca', 'alcachofa', 'zanahoria', 'albahaca', 'calabaza', 'remolacha','salmorejo','sopa', 'gazpacho','patatas', 'patata', 'arroz', 'tomate', 'mozzarella','pollo','proteínas', 'pollo', 'salmón', 'desnatada',  'desnatada', 'desnatada','avena','arroz','0%','integrales','avena','ensalada','alcachofas','champiñones','alcachofa','verduras','verdura', 'pimientos', 'espinacas', 'cebolla','haba', 'rollitos', 'rodajas', 'lechuga', 'iceberg','zanahoria', 'brócoli', 'berenjena','judía','berenjenas', 'coliflor', 'pepino', 'escarola', 'alcachofa','tomate', 'puerro', 'calabaza', 'calabacín','pimiento','patata', 'remolacha', 'zanahoria', 'cebolla', 'ajo','ajos','puerro','maíz','espárrago','puerro','espinaca','rúcula', 'lechuga', 'canónigos', 'coliflor', 'brócoli', 'alcachofa','acelgas','champiñón', 'seta', 'zanahorias', 'tomates', 'cebollas', 'batata', 'lechugas']
prob_3_2_4_entre_horas = ['york','lonchas', 'lonchas''lonchas','pavo', 'pavo','pavo','lonchas']
cluster4 = [prob_3_2_4_desayuno, prob_3_2_4_comida_cena, prob_3_2_4_entre_horas]

In [116]:
def modelo_recomendacion(tipo_comprador):
    
    if tipo_comprador == '1':
        productos = [9,9,3,5,6]
        
        Desayuno, comida_cena, entre_horas, Cuidado_personal, Hogar = set_probabilities(cluster1)
        df_cesta = Desayuno.sample(n=productos[0], weights = Desayuno.prob).append(comida_cena.sample(n=productos[1], weights = comida_cena.prob)).append(entre_horas.sample(n=productos[2], weights = entre_horas.prob))#.append(Hogar.sample(n=productos[3])).append(Cuidado_personal.sample(n=productos[4]))
        lista_productos = df_cesta[' Name'].tolist()
        precio_total = df_cesta['Price'].sum()
    elif tipo_comprador == '2':
        productos = [9,9,7,5,6]
        Desayuno, comida_cena, entre_horas, Cuidado_personal, Hogar = set_probabilities(cluster2)
        df_cesta = Desayuno.sample(n=productos[0], weights = Desayuno.prob).append(comida_cena.sample(n=productos[1], weights = comida_cena.prob)).append(entre_horas.sample(n=productos[2], weights = entre_horas.prob))#.append(Hogar.sample(n=productos[3])).append(Cuidado_personal.sample(n=productos[4]))
        lista_productos = df_cesta[' Name'].tolist()
        precio_total = df_cesta['Price'].sum()
    elif tipo_comprador == '3':
        productos = [9,9,7,5,6]
        Desayuno, comida_cena, entre_horas, Cuidado_personal, Hogar = set_probabilities(cluster3)
        df_cesta = Desayuno.sample(n=productos[0], weights = Desayuno.prob).append(comida_cena.sample(n=productos[1], weights = comida_cena.prob)).append(entre_horas.sample(n=productos[2], weights = entre_horas.prob))#.append(Hogar.sample(n=productos[3])).append(Cuidado_personal.sample(n=productos[4]))
        lista_productos = df_cesta[' Name'].tolist()
        precio_total = df_cesta['Price'].sum()
    elif tipo_comprador == '4':
        productos = [9,9,7,5,6]
        Desayuno, comida_cena, entre_horas, Cuidado_personal, Hogar = set_probabilities(cluster4)
        df_cesta = Desayuno.sample(n=productos[0], weights = Desayuno.prob).append(comida_cena.sample(n=productos[1], weights = comida_cena.prob)).append(entre_horas.sample(n=productos[2], weights = entre_horas.prob))#.append(Hogar.sample(n=productos[3])).append(Cuidado_personal.sample(n=productos[4]))
        lista_productos = df_cesta[' Name'].tolist()
        precio_total = df_cesta['Price'].sum()
    return lista_productos, precio_total
